<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/downsampling_bagging0823%20git22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [118]:
# LightGBM
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler


In [119]:
#control
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas_profiling
import seaborn as sns
from pandas_profiling.utils.cache import cache_file

#Optuna
!pip install optuna
import optuna
from sklearn.metrics import log_loss

#何で必要なのか分かってない
%matplotlib inline
warnings.filterwarnings('ignore')

  Cloning https://github.com/pandas-profiling/pandas-profiling.git to /tmp/pip-req-build-2jrywfoy
  Running command git clone -q https://github.com/pandas-profiling/pandas-profiling.git /tmp/pip-req-build-2jrywfoy
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=7894e0b671b90e2b014e2f198bdf8d12327516108dc4f2a6afb74b5e1d290c70
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhtyh683/wheels/cd/13/75/8fcecd52c706914e90d916ede57f8c74de60e4c9ebc6c2f3b7
Successfully built pandas-profiling
  Using cached https://files.pythonhosted.org/packages/06/b0/9a6313c78bca92abfacc08a2ad8b27bfe845256f615786ee2b6452ae1978/optuna-2.0.0.tar.gz
     |████████████████████████████████| 1.1MB 2.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |█████████

In [308]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data		       submit_sample.csv  train.csv
submission_lightgbm_kfold.csv  test.csv


In [121]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

MessageError: ignored

In [309]:
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["income"]=data["job"].copy()
data["goodincome"]=data["job"].copy()
data["job"].replace(['blue-collar','management','technician','admin.','services',\
                     'unknown','self-employed','entrepreneur','student','retired',\
                     'unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["married"]=data["marital"]
data["single"]=data["marital"]
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,1,0,2],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,1,2,0],inplace=True)
#data["job"] = data["job"].astype(np.int64)

data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],\
                      [3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)
#data["y"].astype(np.int64)
#ここから追加
data["year"]=data["month"]*30+data["day"]
data["campaign+previous"]=data["campaign"] + data["previous"]
data["poutcome/pdays"]=data["poutcome"] / (data["pdays"]+10) * 10000
data["poutcome/pdays"] = data["poutcome/pdays"].astype(np.int64)
data["income"].replace(['blue-collar','management','technician',\
                                         'admin.','services','unknown','self-employed',\
                                         'entrepreneur','student','retired','unemployed','housemaid'],\
                                        [0,0,0,0,0,0,0,0,1,1,1,0],inplace=True)
data["goodincome"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],\
                                                [0,1,0,1,0,0,1,1,0,0,0,0],inplace=True)
data["housing+loan"] = data["housing"] + data["loan"]
data["duration*campaign"] = data["duration"] * data["campaign"] 
data["married"].replace(['married','single','divorced'],[1,0,1],inplace=True)
data["single"].replace(['married','single','divorced'],[1,0,0],inplace=True)
train["y"]=train["y"].astype(np.int64)

#全体のうち、残したいモノだけ選ぶ
all_columns = list(train.columns)
remain_columns = ['age','job','education','balance','housing','loan',
                  'contact','day','month','duration','campaign','pdays','previous',
                  'poutcome','y','income','goodincome',"married","single","year",
                  'campaign+previous'	,'poutcome/pdays'	,'housing+loan','duration*campaign']
                  
delete_columns = list(set(all_columns)-set(remain_columns))
data.drop(delete_columns, axis=1, inplace=True)

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]

#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y = train['y']
X = train.drop('y', axis = 1)
X_sub = test.drop('y', axis = 1)
#Xcolumns=X_train.columns

In [310]:
def imbalanced_data_split(X, y, test_size=0.2):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        return X_train, X_test, y_train, y_test

In [311]:
X_train, X_test, y_train, y_test = imbalanced_data_split(X.values, y.values, test_size=0.2)
X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_train, y_train, test_size=0.2)

# LightGBM

In [312]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': 40,
    'boosting_type' : 'gbdt',
    'reg_alpha' : 1,
    'reg_lambda' : 1,
    'objective': 'binary',
    'max_bin': 300,
    'metric': 'auc',
}

#categorical_features = ['job','education','housing','loan','contact','poutcome',"income",'goodincome','married',"single",'housing+loan']
#X_train[categorical_features]=X_train[categorical_features].astype('category')

In [313]:
def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

# Imbalanced-learn

In [314]:
sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)

In [315]:
%%time
model_under_sample = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)

[1]	valid_0's auc: 0.782838
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.783844
[3]	valid_0's auc: 0.787057
[4]	valid_0's auc: 0.790619
[5]	valid_0's auc: 0.793744
[6]	valid_0's auc: 0.79669
[7]	valid_0's auc: 0.798335
[8]	valid_0's auc: 0.799784
[9]	valid_0's auc: 0.800313
[10]	valid_0's auc: 0.801469
[11]	valid_0's auc: 0.802383
[12]	valid_0's auc: 0.802952
[13]	valid_0's auc: 0.801346
[14]	valid_0's auc: 0.800939
[15]	valid_0's auc: 0.802423
[16]	valid_0's auc: 0.802283
[17]	valid_0's auc: 0.803136
[18]	valid_0's auc: 0.803788
[19]	valid_0's auc: 0.80385
[20]	valid_0's auc: 0.804099
[21]	valid_0's auc: 0.804742
[22]	valid_0's auc: 0.804598
[23]	valid_0's auc: 0.805136
[24]	valid_0's auc: 0.80581
[25]	valid_0's auc: 0.806038
[26]	valid_0's auc: 0.806388
[27]	valid_0's auc: 0.806169
[28]	valid_0's auc: 0.807373
[29]	valid_0's auc: 0.808428
[30]	valid_0's auc: 0.808664
[31]	valid_0's auc: 0.809369
[32]	valid_0's auc: 0.809193
[33]	valid_0's auc: 0.

In [316]:
# テストデータを予測する
y_pred_under_sample = model_under_sample.predict(X_test, num_iteration=model_under_sample.best_iteration)

# auc を計算する
auc = roc_auc_score(y_test, y_pred_under_sample)
print(auc)

0.8430898425721


# Bagging

In [317]:
def bagging(seed):
    sampler = RandomUnderSampler(random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)
    model_bagging = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)
    return model_bagging

In [318]:
%%time
models = []

for i in range(10):
    models.append(bagging(i))

[1]	valid_0's auc: 0.800335
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.798519
[3]	valid_0's auc: 0.799272
[4]	valid_0's auc: 0.798983
[5]	valid_0's auc: 0.798217
[6]	valid_0's auc: 0.805683
[7]	valid_0's auc: 0.805727
[8]	valid_0's auc: 0.810156
[9]	valid_0's auc: 0.812143
[10]	valid_0's auc: 0.811465
[11]	valid_0's auc: 0.812218
[12]	valid_0's auc: 0.812668
[13]	valid_0's auc: 0.812787
[14]	valid_0's auc: 0.812524
[15]	valid_0's auc: 0.812546
[16]	valid_0's auc: 0.814533
[17]	valid_0's auc: 0.815767
[18]	valid_0's auc: 0.816783
[19]	valid_0's auc: 0.817684
[20]	valid_0's auc: 0.816577
[21]	valid_0's auc: 0.817124
[22]	valid_0's auc: 0.817216
[23]	valid_0's auc: 0.817531
[24]	valid_0's auc: 0.818113
[25]	valid_0's auc: 0.818376
[26]	valid_0's auc: 0.818726
[27]	valid_0's auc: 0.819443
[28]	valid_0's auc: 0.81912
[29]	valid_0's auc: 0.819146
[30]	valid_0's auc: 0.819452
[31]	valid_0's auc: 0.820061
[32]	valid_0's auc: 0.819842
[33]	valid_0's auc: 

In [319]:
y_preds = []

for m in models:
    y_preds.append(m.predict(X_test, num_iteration=m.best_iteration))

y_preds_bagging = sum(y_preds)/len(y_preds)
# auc を計算する
auc = roc_auc_score(y_test, y_preds_bagging)
print(auc)

for y_pred in y_preds:
    print(roc_auc_score(y_test, y_pred))

0.8476618135405821
0.8351283641418653
0.8422176927643095
0.8341926818120613
0.8220371086823356
0.8392314271680235
0.8371068806669588
0.837431181003207
0.8379342258230306
0.8193692168810652
0.8358305407471045


In [320]:
y_subs = []
percentage=0.32

for m in models:
    y_subs.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging = sum(y_subs)/len(y_subs)
y_sub = (y_subs_bagging > percentage).astype(int)

sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(9611, 18050)

In [321]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Optuna

In [322]:
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.1,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [323]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.194557	valid_1's binary_logloss: 0.884489
[20]	training's binary_logloss: 0.168532	valid_1's binary_logloss: 0.755621
[30]	training's binary_logloss: 0.150763	valid_1's binary_logloss: 0.663978
[40]	training's binary_logloss: 0.135896	valid_1's binary_logloss: 0.587268
[50]	training's binary_logloss: 0.123953	valid_1's binary_logloss: 0.53141
[60]	training's binary_logloss: 0.114119	valid_1's binary_logloss: 0.483016
[70]	training's binary_logloss: 0.105829	valid_1's binary_logloss: 0.441635
[80]	training's binary_logloss: 0.0984376	valid_1's binary_logloss: 0.402197
[90]	training's binary_logloss: 0.0914537	valid_1's binary_logloss: 0.371824
[100]	training's binary_logloss: 0.0852107	valid_1's binary_logloss: 0.343393
[110]	training's binary_logloss: 0.0792592	valid_1's binary_logloss: 0.315768
[120]	training's binary_logloss: 0.0740323	valid_1's binary_logloss: 0.292822
[130]	training's bi

[I 2020-08-23 04:06:10,194] Trial 0 finished with value: 0.001226338946750404 and parameters: {'max_bin': 427, 'num_leaves': 79}. Best is trial 0 with value: 0.001226338946750404.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.189629	valid_1's binary_logloss: 0.859781
[20]	training's binary_logloss: 0.161884	valid_1's binary_logloss: 0.722852
[30]	training's binary_logloss: 0.141819	valid_1's binary_logloss: 0.624778
[40]	training's binary_logloss: 0.125536	valid_1's binary_logloss: 0.541075
[50]	training's binary_logloss: 0.112451	valid_1's binary_logloss: 0.470591
[60]	training's binary_logloss: 0.102097	valid_1's binary_logloss: 0.419232
[70]	training's binary_logloss: 0.0934972	valid_1's binary_logloss: 0.375049
[80]	training's binary_logloss: 0.0863886	valid_1's binary_logloss: 0.339696
[90]	training's binary_logloss: 0.0791508	valid_1's binary_logloss: 0.307136
[100]	training's binary_logloss: 0.072249	valid_1's binary_logloss: 0.276877
[110]	training's binary_logloss: 0.066403	valid_1's binary_logloss: 0.253385
[120]	training's binary_logloss: 0.0612806	valid_1's binary_logloss: 0.233577
[130]	training's bi

[I 2020-08-23 04:06:19,873] Trial 1 finished with value: 0.00027593006196168846 and parameters: {'max_bin': 372, 'num_leaves': 96}. Best is trial 1 with value: 0.00027593006196168846.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.18892	valid_1's binary_logloss: 0.852295
[20]	training's binary_logloss: 0.161275	valid_1's binary_logloss: 0.712446
[30]	training's binary_logloss: 0.140562	valid_1's binary_logloss: 0.597396
[40]	training's binary_logloss: 0.124299	valid_1's binary_logloss: 0.522873
[50]	training's binary_logloss: 0.111022	valid_1's binary_logloss: 0.458422
[60]	training's binary_logloss: 0.100619	valid_1's binary_logloss: 0.407422
[70]	training's binary_logloss: 0.0915552	valid_1's binary_logloss: 0.363601
[80]	training's binary_logloss: 0.0840956	valid_1's binary_logloss: 0.329875
[90]	training's binary_logloss: 0.0768716	valid_1's binary_logloss: 0.30071
[100]	training's binary_logloss: 0.070433	valid_1's binary_logloss: 0.272577
[110]	training's binary_logloss: 0.065071	valid_1's binary_logloss: 0.250921
[120]	training's binary_logloss: 0.0597312	valid_1's binary_logloss: 0.228624
[130]	training's bina

...

[I 2020-08-23 04:11:28,111] Trial 37 finished with value: 0.004405533919380078 and parameters: {'max_bin': 441, 'num_leaves': 63}. Best is trial 7 with value: 3.582210320033969e-05.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.209553	valid_1's binary_logloss: 0.969644
[20]	training's binary_logloss: 0.191821	valid_1's binary_logloss: 0.886662
[30]	training's binary_logloss: 0.180693	valid_1's binary_logloss: 0.822997
[40]	training's binary_logloss: 0.17143	valid_1's binary_logloss: 0.77334
[50]	training's binary_logloss: 0.163816	valid_1's binary_logloss: 0.737628
[60]	training's binary_logloss: 0.157038	valid_1's binary_logloss: 0.703794
[70]	training's binary_logloss: 0.151048	valid_1's binary_logloss: 0.669169
[80]	training's binary_logloss: 0.145473	valid_1's binary_logloss: 0.644128
[90]	training's binary_logloss: 0.1405	valid_1's binary_logloss: 0.621846
[100]	training's binary_logloss: 0.135406	valid_1's binary_logloss: 0.597787
[110]	training's binary_logloss: 0.130623	valid_1's binary_logloss: 0.573798
[120]	training's binary_logloss: 0.126382	valid_1's binary_logloss: 0.551024
[130]	training's binary_log

[I 2020-08-23 04:11:33,987] Trial 38 finished with value: 0.0503241272084663 and parameters: {'max_bin': 375, 'num_leaves': 33}. Best is trial 7 with value: 3.582210320033969e-05.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.196206	valid_1's binary_logloss: 0.892207
[20]	training's binary_logloss: 0.171814	valid_1's binary_logloss: 0.769911
[30]	training's binary_logloss: 0.154608	valid_1's binary_logloss: 0.682787
[40]	training's binary_logloss: 0.139811	valid_1's binary_logloss: 0.611949
[50]	training's binary_logloss: 0.128412	valid_1's binary_logloss: 0.551266
[60]	training's binary_logloss: 0.119468	valid_1's binary_logloss: 0.507475
[70]	training's binary_logloss: 0.11148	valid_1's binary_logloss: 0.467354
[80]	training's binary_logloss: 0.104572	valid_1's binary_logloss: 0.434966
[90]	training's binary_logloss: 0.0985863	valid_1's binary_logloss: 0.406447
[100]	training's binary_logloss: 0.0926481	valid_1's binary_logloss: 0.3804
[110]	training's binary_logloss: 0.0870145	valid_1's binary_logloss: 0.354282
[120]	training's binary_logloss: 0.0817091	valid_1's binary_logloss: 0.331046
[130]	training's binar

[I 2020-08-23 04:11:42,033] Trial 39 finished with value: 0.0019309550479331777 and parameters: {'max_bin': 320, 'num_leaves': 73}. Best is trial 7 with value: 3.582210320033969e-05.


In [324]:
study.best_params

{'max_bin': 471, 'num_leaves': 120}

In [331]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': 120,
    'boosting_type' : 'gbdt',
    'reg_alpha' : 1,
    'reg_lambda' : 1,
    'objective': 'binary',
    'max_bin': 471,
    'metric': 'auc',
    'max_dapth': 6
}

#categorical_features = ['job','education','housing','loan','contact','poutcome',"income",'goodincome','married',"single",'housing+loan']
#X_train[categorical_features]=X_train[categorical_features].astype('category')

In [332]:
%%time
models = []

for i in range(10):
    models.append(bagging(i))

[1]	valid_0's auc: 0.797342
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.794532
[3]	valid_0's auc: 0.794414
[4]	valid_0's auc: 0.793797
[5]	valid_0's auc: 0.795342
[6]	valid_0's auc: 0.797994
[7]	valid_0's auc: 0.80416
[8]	valid_0's auc: 0.808139
[9]	valid_0's auc: 0.806979
[10]	valid_0's auc: 0.807614
[11]	valid_0's auc: 0.808406
[12]	valid_0's auc: 0.808703
[13]	valid_0's auc: 0.809557
[14]	valid_0's auc: 0.809369
[15]	valid_0's auc: 0.809233
[16]	valid_0's auc: 0.809268
[17]	valid_0's auc: 0.809727
[18]	valid_0's auc: 0.810839
[19]	valid_0's auc: 0.811097
[20]	valid_0's auc: 0.811999
[21]	valid_0's auc: 0.811233
[22]	valid_0's auc: 0.810524
[23]	valid_0's auc: 0.811391
[24]	valid_0's auc: 0.811237
[25]	valid_0's auc: 0.811784
[26]	valid_0's auc: 0.812472
[27]	valid_0's auc: 0.813202
[28]	valid_0's auc: 0.813688
[29]	valid_0's auc: 0.814747
[30]	valid_0's auc: 0.815395
[31]	valid_0's auc: 0.815474
[32]	valid_0's auc: 0.815443
[33]	valid_0's auc: 

In [333]:
y_preds = []

for m in models:
    y_preds.append(m.predict(X_test, num_iteration=m.best_iteration))

y_preds_bagging = sum(y_preds)/len(y_preds)
# auc を計算する
auc = roc_auc_score(y_test, y_preds_bagging)
print(auc)

0.8446312174158762


In [334]:
y_subs = []

for m in models:
    y_subs.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging = sum(y_subs)/len(y_subs)
# auc を計算する
y_subs_bagging

array([0.92079642, 0.24738645, 0.19320765, ..., 0.5197584 , 0.08982739,
       0.61945623])

In [335]:
percentage=0.37
y_sub = (y_subs_bagging > percentage).astype(int)

sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(8623, 18050)

In [336]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>